# Azure functions example

Dieses Notebook zeigt, wie man die Funktionsaufrufe mit dem Azure OpenAI Service verwendet. Funktionen ermöglichen es einem Aufrufer von Chatvervollständigungen, Fähigkeiten zu definieren, die das Modell verwenden kann, um seine
Funktionalität in externe Tools und Datenquellen zu erweitern.

Sie können mehr über Chat-Funktionen im OpenAI-Blog lesen: https://openai.com/blog/function-calling-and-other-api-updates


**HINWEIS**: Chat-Funktionen erfordern Modellversionen, die mit den Labels gpt-4 und gpt-35-turbo `-0125` beginnen. Sie werden von älteren Versionen der Modelle nicht unterstützt.

## Setup

Zunächst installieren wir die erforderlichen Abhängigkeiten und importieren die Bibliotheken, die wir verwenden werden.

In [1]:
import os
import openai
import dotenv

dotenv.load_dotenv()

endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
api_key = os.environ["AZURE_OPENAI_API_KEY"]

#### Authentication

In [4]:
client = openai.AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2023-09-01-preview")

## Deployments

In [5]:
deployment = "gpt-35-turbo-0125" 

## Functions

Nachdem die Einrichtung und Authentifizierung abgeschlossen sind, können wir nun Funktionen mit dem Azure OpenAI-Service verwenden. Das ist in folgende Schritte aufgeteilt:

1. Definieren Sie die Funktion(en)
2. Übergabe der Funktionsdefinition(en) an die Chatvervollständigungs-API
3. Aufruf der Funktion mit Argumenten aus der Antwort
4. Rückführung der Funktionsantwort in die Chatvervollständigungs-API


#### 1. Definieren der Funktion(en)

Es kann eine Liste von Funktionen definiert werden, die jeweils den Namen der Funktion, eine optionale Beschreibung und die Parameter, die die Funktion akzeptiert (beschrieben als JSON-Schema), enthalten.

In [7]:
functions = [
        {
            "type": "function",
            "function": {
                "name": "get_current_weather",
                "description": "Get the current weather in a given location",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state, e.g. San Francisco, CA",
                        },
                        "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                    },
                    "required": ["location"],
                },
            },
        }
    ]

#### 2. Übergabe der Funktionsdefinition(en) an die Chatvervollständigungs-API

Jetzt können wir die Funktion an die Chatvervollständigungs-API übergeben. Wenn das Modell beschließt, die Funktion aufzurufen, wird ein `finish_reason` von "tool_calls" in die Auswahl eingefügt und die Details der aufzurufenden Funktion und ihre Argumente werden in der `message` enthalten sein. Optional können Sie das Schlüsselwortargument `tool_choice` setzen, um das Modell zum Aufruf einer bestimmten Funktion zu zwingen (z.B. `{"type": "function", "function": {"name": get_current_weather}}`). Standardmäßig ist dies auf `auto` gesetzt, so dass das Modell selbst entscheiden kann, ob es die Funktion aufrufen will oder nicht. 

In [8]:
messages = [
    {"role": "system", "content": "Stellen keine Vermutungen darüber an, welche Werte Sie in die Funktionen eingeben sollen. Bitten Sie um Klärung, wenn eine Benutzeranfrage mehrdeutig ist."},
    {"role": "user", "content": "Wie ist das Wetter heute in Hamburg?"}
]

chat_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions,
)
print(chat_completion)

ChatCompletion(id='chatcmpl-9AfAGFQplaVjTKKP1SkHh5GRimlSd', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BMDj0T63DKrYECNzANfFhyDe', function=Function(arguments='{"location":"Hamburg"}', name='get_current_weather'), type='function')]), content_filter_results={})], created=1712328544, model='gpt-35-turbo', object='chat.completion', system_fingerprint='fp_2f57f81c11', usage=CompletionUsage(completion_tokens=16, prompt_tokens=125, total_tokens=141), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}])


In [9]:
chat_completion.choices

[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_BMDj0T63DKrYECNzANfFhyDe', function=Function(arguments='{"location":"Hamburg"}', name='get_current_weather'), type='function')]), content_filter_results={})]

#### 3. Aufruf einer Funktion mit den Argumenten aus der Antwort

Der Name des Funktionsaufrufs ist derjenige, der ursprünglich angegeben wurde, und die Argumente enthalten JSON, das dem in der Funktionsdefinition enthaltenen Schema entspricht.

In [10]:
import json

def get_current_weather(request):
    """
    Demo function - do something useful here.
    """
    location = request.get("location")
    unit = request.get("unit")
    return {"temperature": "22", "unit": "celsius", "description": "Wolkig mit der aussicht auf Fleischbällchen"}

function_call = chat_completion.choices[0].message.tool_calls[0].function
print(function_call.name)
print(function_call.arguments)

if function_call.name == "get_current_weather":
    response = get_current_weather(json.loads(function_call.arguments))

get_current_weather
{"location":"Hamburg"}


#### 4. Feed function response back into chat completions API

The response from the function should be serialized into a new message with the role set to "function". Now the model will use the response data to formulate its answer.

In [11]:
messages.append(
    {
        "role": "function",
        "name": "get_current_weather",
        "content": json.dumps(response)
    }
)

function_completion = client.chat.completions.create(
    model=deployment,
    messages=messages,
    tools=functions,
)

print(function_completion.choices[0].message.content.strip())

Das Wetter heute in Hamburg ist wolkig mit der Aussicht auf Fleischbällchen. Die Temperatur beträgt 22 Grad Celsius.
